## 2. Clasificación con SVM

Como se ha mencionado en el preprocesamiento, en el problema que aquí se estudia, cada individuo puede pertenecer a una clase, las dos o ninguna, por lo que se está tratando de un problema multietiqueta. Para usar una máquina de soporte vectorial en clasificación es necesario codificar la salida para que las distintas variantes de SVM pueda realizar la clasificación.

In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.svm import LinearSVC, SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier



In [2]:
X_train = pd.read_csv("preprocessed_X_train.csv", index_col="respondent_id")
y_train = pd.read_csv("preprocessed_y_train.csv", index_col="respondent_id")

# Cargar el dataset de prueba
X_test = pd.read_csv("preprocessed_X_test.csv", index_col="respondent_id")
y_test = pd.read_csv("preprocessed_y_test.csv", index_col="respondent_id")

In [27]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape, y_test.shape)


Train set: (20739, 96) (20739, 2)
Test set: (5195, 96) (5195, 2)


In [28]:
X_test.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
respondent_id,,,,,,,,,,,,,,,,,,,,,
22317,1.0,1.0,0,1,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1215,2.0,2.0,0,1,0,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10540,2.0,1.0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7407,3.0,1.0,0,1,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14697,1.0,1.0,0,1,0,1,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_train.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
respondent_id,,,,,,,,,,,,,,,,,,,,,
12230,2.0,2.0,0,1,0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8871,1.0,1.0,0,0,0,1,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2390,1.0,1.0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9227,2.0,1.0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9064,2.0,1.0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Antes de proceder, es necesario escalar los valores, sobre todo para que las variables de mayor rango (pertenencientes a las preguntas de opinión mayoritariamente) no influyan en el cálculo de las distancias con SVM. Dentro de las posibilidades de escalado, se puede optar por:
- Normalización zero-mean: Escala los valores en un rango $[0,1]$ en función de la media y la varianza de la variable. Esta normalización se usa cuando las distribuciones de las variables siguen aproximadamente una distribución normal.
- Normalización MinMaxScaling: Escala los valores en un rango $[0,1]$ en función de los valores máximos y mínimos de cada variable. Esta técnica de escalado conserva las características de los datos originales tras la transformación. 

Dado que no se conocen las distribuciones originales de los datos, se va a optar por el uso de escalado MinMax

In [3]:
# Escalar características
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[0.25      , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.25      , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 0.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.25      , 0.33333333, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [4]:
# Asegurarse de que todos los valores sean numéricos
#X_train = X_train.astype(float)
#X_test = X_test.astype(float)
y_train = y_train.astype(int)

from sklearn.model_selection import train_test_split

X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    X_train_scaled, y_train, test_size=0.33, random_state=42
)

Una vez tenemos todos los datos en el mismo formato, podemos pasar a implementar SVM. Este algoritmo trata de sobredimensionar los datos para poder generar un hiperplano de separación entre dos clases, por lo que, de manera nativa no es capaz de realizar una clasificación multietiqueta. Para solventar este problema se usa un pequeño 'truco' a la hora de implementar el algoritmo, que es la descomposición binaria de la respuesta obtenida, de esta manera SVM puede abordar los problemas multietiqueta como problemas independientes:

- Si se entrena un clasificador para cada par de valores posibles se está usando una estrategia OVO (One-vs-One)
- Si el clasificador es capaz de distinguir en cada caso su etiqueta frente a las demás se usa una estrategia OVR (One-vs-Rest)

Otra opción para la clasificación multietiqueta pasa por el uso de la función 'MultiOutputClassifier' implementada en 'sklearn', capaz de tratar cada etiqueta como un problema binario independiente, entrenando un clasificador para cada etiqueta sin comparar con las demás:


#### Clasificación SVM con RBF y balanceo de clases

##### Optimización por GridSearch y clases balanceadas

In [ ]:
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Paso 2: Configurar MultiOutputClassifier para manejar multietiqueta
multi_label_svm = MultiOutputClassifier(base_svm)

# Paso 3: Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'estimator__C': [0.1, 1, 10],           # Regularización
    'estimator__gamma': ['scale', 'auto'],  # Parámetro del kernel RBF
    'estimator__max_iter': [10000, 50000],  # Iteraciones máximas
    'estimator__kernel': ['rbf']            # Kernel
}

# Paso 4: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=multi_label_svm,
    param_grid=param_grid,
    cv=3,  # Validación cruzada de 5 pliegues
    scoring='roc_auc',  # Métrica para evaluar el modelo
    verbose=3,  # Para ver el progreso
    n_jobs=-1,   # Usar todos los núcleos disponibles
    refit=True

)

# Paso 5: Entrenar con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Paso 6: Resultados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en optimización (train): {grid_search.best_score_}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))
roc_auc = roc_auc_score(y_test, y_valid_pred, average='macro', multi_class='ovr')

# Imprimir el resultado
print(f"ROC AUC Score (test): {roc_auc:.4f}")


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Mejores parámetros encontrados:  {'estimator__C': 10, 'estimator__gamma': 'auto', 'estimator__kernel': 'rbf', 'estimator__max_iter': 50000}
Accuracy en validación: 0.6669874879692012
ROC AUC Score en optimización (train): 0.8600943278432579
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.54      0.77      0.64      1106
           1       0.77      0.79      0.78      2430

   micro avg       0.68      0.78      0.73      3536
   macro avg       0.65      0.78      0.71      3536
weighted avg       0.70      0.78      0.73      3536
 samples avg       0.37      0.39      0.37      3536

ROC AUC Score (test): 0.7940


En este caso buscaremos los valores optimos para `gamma`= ['auto','scale'] con random search y veremos si los resultados y los tiempos de procesamiento mejoran:

In [33]:
# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Configurar MultiOutputClassifier
multi_label_svm = MultiOutputClassifier(base_svm)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': np.logspace(-3, 1, 5),  # Ajuste más fino para 'C'
    'estimator__gamma': ["scale", "auto"], 
    'estimator__max_iter': [10000, 50000, 100000],
    'estimator__kernel': ['rbf']
}



# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__max_iter': 50000, 'estimator__kernel': 'rbf', 'estimator__gamma': 'auto', 'estimator__C': np.float64(10.0)}
Mejor score en train: 0.8600943278432579
Accuracy en validación: 0.6669874879692012
ROC AUC Score en Test: 0.7940
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.54      0.77      0.64      1106
           1       0.77      0.79      0.78      2430

   micro avg       0.68      0.78      0.73      3536
   macro avg       0.65      0.78      0.71      3536
weighted avg       0.70      0.78      0.73      3536
 samples avg       0.37      0.39      0.37      3536



Podemos apreciar que tanto para un optimizador como el otro obtienen los mismos resultados tanto en las métricas de evaluación como en el reporte. Al usar `GridSearch` estamos buscando la optimización en un conjunto de valores definidos, lo que produce una búsqueda exhaustiva de parámetros para SVM en menor tiempo que RandomSearch, pero también menor rango de parámetros. En este caso, RandomSearch parece ser de mayor utilidad, ya que es capaz de explorar más opciones que un GridSearch simple con un tiempo computacional similar. 

In [11]:
# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Configurar MultiOutputClassifier
multi_label_svm = MultiOutputClassifier(base_svm)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.1, 1, 10],  # Ajuste más fino para 'C'
    'estimator__gamma': np.logspace(-2, 0, 3), 
    'estimator__max_iter': [10000, 50000, 100000],
    'estimator__kernel': ['rbf']
}



# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Mejores parámetros: {'estimator__max_iter': 10000, 'estimator__kernel': 'rbf', 'estimator__gamma': np.float64(0.01), 'estimator__C': 10}
Mejor score en train: 0.8600857727001766


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6548604427333975
ROC AUC Score en Test: 0.7907
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.52      0.78      0.63      1106
           1       0.76      0.80      0.78      2430

   micro avg       0.67      0.79      0.72      3536
   macro avg       0.64      0.79      0.70      3536
weighted avg       0.68      0.79      0.73      3536
 samples avg       0.37      0.39      0.37      3536



En este caso, parece que RandomSearch no ha sido de tanta utilidad, ya que no se ha conseguido una solución óptima. Volvemos a probar con GridSearch para comprobar si este es capaz de proporcionarnos mejores resultados:

In [35]:
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Paso 2: Configurar MultiOutputClassifier para manejar multietiqueta
multi_label_svm = MultiOutputClassifier(base_svm)

# Paso 3: Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'estimator__C': [0.1, 1, 10],           # Regularización
    'estimator__gamma': np.logspace(-2, 0, 3),  # Parámetro del kernel RBF
    'estimator__max_iter': [10000, 50000],  # Iteraciones máximas
    'estimator__kernel': ['rbf']            # Kernel
}

# Paso 4: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=multi_label_svm,
    param_grid=param_grid,
    cv=3,  # Validación cruzada de 5 pliegues
    scoring='roc_auc',  # Métrica para evaluar el modelo
    verbose=3,  # Para ver el progreso
    n_jobs=-1,   # Usar todos los núcleos disponibles
    refit=True

)

# Paso 5: Entrenar con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Paso 6: Resultados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en optimización (train): {grid_search.best_score_}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))
roc_auc = roc_auc_score(y_test, y_valid_pred)

# Imprimir el resultado
print(f"ROC AUC Score (test): {roc_auc:.4f}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros encontrados:  {'estimator__C': 10, 'estimator__gamma': np.float64(0.01), 'estimator__kernel': 'rbf', 'estimator__max_iter': 50000}
Accuracy en validación: 0.668334937439846
ROC AUC Score en optimización (train): 0.8601681856859296
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.54      0.77      0.64      1106
           1       0.77      0.79      0.78      2430

   micro avg       0.68      0.78      0.73      3536
   macro avg       0.66      0.78      0.71      3536
weighted avg       0.70      0.78      0.73      3536
 samples avg       0.37      0.39      0.37      3536

ROC AUC Score (test): 0.7947


A diferencia de los resultados obtenidos con RandomSearch, reduciendo las opciones de iteraciones máximas y haciendo una búsqueda GridSearch, sí que se han conseguido mejorar los resultados iniciales (cambiando el parámetro `gamma` a numérico). Obviamente, si a la optimización GridSearch se le añaden más parámetros la búsqueda será muy lenta, por lo que seguimos probando otras formas de conseguir los parámetros de SVM óptimos.

Vamos a comprobar si asignar pesos distintos a las clases (`class_weight`= Balanced) una vez los datos han sido estratificados puede perjudicar al SVM:

In [36]:
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)

# Paso 2: Configurar MultiOutputClassifier para manejar multietiqueta
multi_label_svm = MultiOutputClassifier(base_svm)

# Paso 3: Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'estimator__C': [0.1, 1, 10],           # Regularización
    'estimator__gamma': np.logspace(-2, 0, 3),  # Parámetro del kernel RBF
    'estimator__max_iter': [10000, 50000],  # Iteraciones máximas
    'estimator__class_weight': ['balanced', None]            # Kernel
}

# Paso 4: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=multi_label_svm,
    param_grid=param_grid,
    cv=3,  # Validación cruzada de 5 pliegues
    scoring='roc_auc',  # Métrica para evaluar el modelo
    verbose=3,  # Para ver el progreso
    n_jobs=-1,   # Usar todos los núcleos disponibles
    refit=True

)

# Paso 5: Entrenar con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Paso 6: Resultados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en optimización (train): {grid_search.best_score_}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))
roc_auc = roc_auc_score(y_test, y_valid_pred)

# Imprimir el resultado
print(f"ROC AUC Score (test): {roc_auc:.6f}")


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Mejores parámetros encontrados:  {'estimator__C': 10, 'estimator__class_weight': 'balanced', 'estimator__gamma': np.float64(0.01), 'estimator__max_iter': 50000}
Accuracy en validación: 0.668334937439846
ROC AUC Score en optimización (train): 0.8601681856859296
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.54      0.77      0.64      1106
           1       0.77      0.79      0.78      2430

   micro avg       0.68      0.78      0.73      3536
   macro avg       0.66      0.78      0.71      3536
weighted avg       0.70      0.78      0.73      3536
 samples avg       0.37      0.39      0.37      3536

ROC AUC Score (test): 0.794679


Podemos determinar que, el modelo SVC con kernel no lineal óptimo viene dado por `kernel= rbf`, `gamma=0.01`, `C= 10` y `max_iter= 50000`, con un rendimiento para la métrica roc-auc de $0.7947$ para el test. En este caso, donde el kernel es no lineal, vemos que la optimización de parámetros mediante GridSearch funciona algo mejor


#### SVM Kernel Lineal y Balanceo de Clases

También podemos probar LinearSVC, en este caso el clasificador SVM que se usa kernel lineal y al no obtener probabilidades, no podemos calcular el roc_auc para evaluar el randomsearch:

In [29]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced'))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'estimator__max_iter': [10000, 20000, 50000],  # Opciones discretas
    'estimator__dual': [True, False]  # Opciones binarias
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='hamming',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.01, 'estimator__max_iter': 10000, 'estimator__dual': False, 'estimator__C': 0.1}
Accuracy en validación: 0.6590952839268528
ROC AUC Score en Test: 0.7906
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.53      0.78      0.63      1106
           1       0.76      0.78      0.77      2430

   micro avg       0.67      0.78      0.72      3536
   macro avg       0.65      0.78      0.70      3536
weighted avg       0.69      0.78      0.73      3536
 samples avg       0.36      0.38      0.36      3536



Dada la rapidez del algoritmo en este caso nos podemos permitir una búsqueda más amplia de parámetros intentando alcanzar los resultados cuando se usa el kernel RBF. 
En este caso vamos a determinar también si el modelo admite mejor un balanceo de clases o no:

In [34]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced'))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10, 100],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia
    'estimator__max_iter': [ 5000, 10000, 50000],  # Iteraciones máximas
    'estimator__dual': [True, False],  # Resolución del problema dual (depende de n_features vs. n_samples)
    'estimator__class_weight': ['balanced', None],  # Pesos para las clases
    'estimator__fit_intercept': [True, False],  # Incluir intercepto en el modelo
    'estimator__intercept_scaling': [0.1, 1, 10]  # Escalado del término independiente
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.001, 'estimator__max_iter': 50000, 'estimator__intercept_scaling': 0.1, 'estimator__fit_intercept': False, 'estimator__dual': False, 'estimator__class_weight': None, 'estimator__C': 100}
Accuracy en validación: 0.685851780558229
ROC AUC Score en Test: 0.7508
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.70      0.48      0.57      1106
           1       0.78      0.76      0.77      2430

   micro avg       0.76      0.68      0.72      3536
   macro avg       0.74      0.62      0.67      3536
weighted avg       0.76      0.68      0.71      3536
 samples avg       0.36      0.34      0.34      3536



En este caso vemos que no se ha alcanzado el valor óptimo para roc_auc (SVM con kernel RBF llega hasta 0.7947). De hecho, al incluir más parámetros para optimizar, el modelo que se ha obtenido rinde por debajo de lo esperado, sobre todo comparándolos con la primera prueba de `LinearSVC`, ya que, aunque la métrica de rendimiento es mayor, en general tiene peores valores de auc.

El mejor modelo SVC viene dado por {'estimator__C': 10, 'estimator__class_weight': 'balanced', 'estimator__gamma': np.float64(0.01), 'estimator__max_iter': 50000, random_state=42, kernel='rbf'}, rindiendo por encima del 79.4% en la métrica de evaluación para el conjunto de test (roc_auc). 

Por otro lado, el mejor modelo con Kernel Lineal viene dado por: {'estimator__tol': 0.01, 'estimator__max_iter': 10000, 'estimator__dual': False, 'estimator__C': 0.1, random_state=42}

#### SVM Kernel Lineal y Desbalanceo de Clases

In [ ]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'estimator__max_iter': [10000, 20000, 50000],  # Opciones discretas
    'estimator__dual': [True, False]  # Opciones binarias
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.001, 'estimator__max_iter': 50000, 'estimator__dual': True, 'estimator__C': 10}
Accuracy en validación: 0.685851780558229
ROC AUC Score en Test: 0.7506
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.70      0.48      0.57      1106
           1       0.78      0.76      0.77      2430

   micro avg       0.76      0.68      0.72      3536
   macro avg       0.74      0.62      0.67      3536
weighted avg       0.76      0.68      0.71      3536
 samples avg       0.36      0.34      0.34      3536



Si que se aprecia bastante la rapidez del algoritmo y la optimización con kernel Lineal, por lo que se podría optar por ampliar el rango de parámentros, ya que, de momento, el primer SVM optimizado consigue unos resultados ligeramente superiores (a un coste computacional elevado).

#### One Vs One y One vs Rest

Comparamos los resultados con los que se obtienen a partir de OVO y OVR:

In [37]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

ovr = OneVsRestClassifier(LinearSVC(C=0.1, random_state=42, dual=False, max_iter=10000, class_weight='balanced', tol= 0.01)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_scaled, y_train)
y_pred = ovr.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_test, y_pred, zero_division=0))


Accuracy en validación: 0.6590952839268528
ROC AUC Score: 0.7906
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.53      0.78      0.63      1106
           1       0.76      0.78      0.77      2430

   micro avg       0.67      0.78      0.72      3536
   macro avg       0.65      0.78      0.70      3536
weighted avg       0.69      0.78      0.73      3536
 samples avg       0.36      0.38      0.36      3536



In [38]:
ovr = OneVsRestClassifier(SVC(kernel='rbf',  random_state=42, max_iter=50000, class_weight='balanced', C=10, gamma=0.01)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_scaled, y_train)
y_pred = ovr.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_test, y_pred, zero_division=0))

Accuracy en validación: 0.668334937439846
ROC AUC Score: 0.7947
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.54      0.77      0.64      1106
           1       0.77      0.79      0.78      2430

   micro avg       0.68      0.78      0.73      3536
   macro avg       0.66      0.78      0.71      3536
weighted avg       0.70      0.78      0.73      3536
 samples avg       0.37      0.39      0.37      3536



Parece claro que el método OVR con LinearSVC se acerca en mayor medida a nuestros resultados con MultiOutputClassifier. 

---
---
#### Simplificación del Problema
---
---

Como hemos mencionado anteriormente, SVM no trabaja directamente con problemas multietiqueta, por lo que se envuelve con MultiOutputClassifier. Otra opción válida es dividir inicialmente las etiquetas para descomponerlo en dos SVM distintos:

In [16]:
y=y_train["seasonal_vaccine"]
z=y_train["h1n1_vaccine"]

---
---
#### Vacuna estacional
---
---

In [17]:
x_train=X_train_scaled
x_test= X_test_scaled

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))


##### Sin balanceo

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'max_iter': 50000, 'gamma': np.float64(0.01), 'C': np.float64(10.0)}
Mejor score en train: 0.856263587794837
Accuracy en validación: 0.7874879692011549
ROC AUC Score en Test: 0.7856
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.79      0.82      0.80      2765
           1       0.78      0.76      0.77      2430

    accuracy                           0.79      5195
   macro avg       0.79      0.79      0.79      5195
weighted avg       0.79      0.79      0.79      5195



---
---
#### LinearSVM con balanceo

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced')

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.001, 'max_iter': 10000, 'dual': True, 'C': 0.1}
Mejor score en train: 0.8551257626816358
Accuracy en validación: 0.784985563041386
ROC AUC Score en Test: 0.7849
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.80      0.79      0.80      2765
           1       0.76      0.78      0.77      2430

    accuracy                           0.78      5195
   macro avg       0.78      0.78      0.78      5195
weighted avg       0.79      0.78      0.79      5195



#### LinearSVM Sin balanceo

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.01, 'max_iter': 10000, 'dual': True, 'C': 0.1}
Mejor score en train: 0.8551081154602294
Accuracy en validación: 0.7890279114533205
ROC AUC Score en Test: 0.7874
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.80      0.81      0.80      2765
           1       0.78      0.76      0.77      2430

    accuracy                           0.79      5195
   macro avg       0.79      0.79      0.79      5195
weighted avg       0.79      0.79      0.79      5195



---
---
#### Vacuna H1N1. Clases balanceadas
---
---

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Mejores parámetros: {'max_iter': 10000, 'gamma': np.float64(0.01), 'C': np.float64(10.0)}
Mejor score en train: 0.8642173953392188


c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6569778633301251
ROC AUC Score en Test: 0.6459
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.64      0.82      0.72      2765
           1       0.70      0.47      0.56      2430

    accuracy                           0.66      5195
   macro avg       0.67      0.65      0.64      5195
weighted avg       0.67      0.66      0.65      5195



#### Sin balanceo

In [22]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'max_iter': 50000, 'gamma': np.float64(0.01), 'C': np.float64(10.0)}
Mejor score en train: 0.8622078472502658
Accuracy en validación: 0.6125120307988451
ROC AUC Score en Test: 0.5889
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.58      0.95      0.72      2765
           1       0.81      0.22      0.35      2430

    accuracy                           0.61      5195
   macro avg       0.70      0.59      0.54      5195
weighted avg       0.69      0.61      0.55      5195



---
---
#### LinearSVC balanceado

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced')

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.01, 'max_iter': 10000, 'dual': False, 'C': 0.1}
Mejor score en train: 0.8614179069356479
Accuracy en validación: 0.6581328200192493
ROC AUC Score en Test: 0.6466
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.64      0.83      0.72      2765
           1       0.70      0.47      0.56      2430

    accuracy                           0.66      5195
   macro avg       0.67      0.65      0.64      5195
weighted avg       0.67      0.66      0.65      5195



#### Sin Balanceo

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'tol': 0.01, 'max_iter': 10000, 'dual': True, 'C': 0.1}
Mejor score en train: 0.8614760056185807
Accuracy en validación: 0.6173243503368624
ROC AUC Score en Test: 0.5949
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.59      0.94      0.72      2765
           1       0.79      0.25      0.38      2430

    accuracy                           0.62      5195
   macro avg       0.69      0.59      0.55      5195
weighted avg       0.68      0.62      0.56      5195



# Ensembles


### Para LinearSVM con Balanceo entre clases

In [50]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.multioutput import MultiOutputClassifier


# 1. Modelo base SVM con MultiOutputClassifier para multietiqueta
base_svm = LinearSVC(C=0.1, random_state=42, dual=False, max_iter=10000, class_weight='balanced', tol= 0.01)

# 2. Bagging clásico con SVM
bagging_svm = BaggingClassifier(
    estimator=base_svm,
    n_estimators=20,  # 10 modelos base
    max_samples= 1.0,   # Usa el 100% de las muestras
    max_features=0.75,  # Usa el 75% de las características
    bootstrap=True,
    random_state=42
)
# Envolvemos el bagging con MultiOutputClassifier
bagging_svm_multi = MultiOutputClassifier(bagging_svm)


# 3. Random Subspaces (Bagging con muestreo de características)
random_subspaces = BaggingClassifier(
    estimator=base_svm,
    n_estimators=20,  # 10 modelos base
    max_samples=0.9,   # Usa el 100% de las muestras
    max_features=0.5,  # Usa el 50% de las características
    bootstrap=False,
    random_state=42
)
# Envolvemos el Random Subspaces con MultiOutputClassifier
random_subspaces_multi = MultiOutputClassifier(random_subspaces)

# Lista de modelos para comparar
models = {
    "Bagging_SVM": bagging_svm_multi,
    "Random_Subspaces": random_subspaces_multi
}

# Entrenamiento y evaluación
results = {}
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train_scaled, y_train)

    # Predicción
    y_pred = model.predict(X_test_scaled)

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Guardar resultados
    results[name] = {"accuracy": accuracy, "report": report}
    print(f"=== {name} ===")
    print(f"Accuracy: {accuracy}")
    print(f"Roc-AUC: {roc_auc}")
    print("Classification Report:")
    print(report)

=== Bagging_SVM ===
Accuracy: 0.6581328200192493
Roc-AUC: 0.7879890934618913
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.77      0.63      1106
           1       0.77      0.77      0.77      2430

   micro avg       0.68      0.77      0.72      3536
   macro avg       0.65      0.77      0.70      3536
weighted avg       0.70      0.77      0.73      3536
 samples avg       0.35      0.38      0.36      3536

=== Random_Subspaces ===
Accuracy: 0.6461982675649663
Roc-AUC: 0.7821297895047525
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.77      0.63      1106
           1       0.76      0.75      0.76      2430

   micro avg       0.67      0.76      0.71      3536
   macro avg       0.65      0.76      0.69      3536
weighted avg       0.69      0.76      0.72      3536
 samples avg       0.34      0.37      0.35      3536



#### Ensemble de SVM con kernel RBF

In [49]:

# 1. Modelo base SVM con MultiOutputClassifier para multietiqueta
base_svm = SVC(kernel='rbf',  random_state=42, max_iter=50000, class_weight='balanced', C=10, gamma=0.01)

# 2. Bagging clásico con SVM
bagging_svm = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=0.8,   # Usa el 80% de las muestras
    max_features=1.0,  # Usa el 100% de las características
    bootstrap=True,
    random_state=42
)
# Envolvemos el bagging con MultiOutputClassifier
bagging_svm_multi = MultiOutputClassifier(bagging_svm)


# 3. Random Subspaces (Bagging con muestreo de características)
random_subspaces = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=1.0,   # Usa el 100% de las muestras
    max_features=0.5,  # Usa el 50% de las características
    bootstrap=False,
    random_state=42
)
# Envolvemos el Random Subspaces con MultiOutputClassifier
random_subspaces_multi = MultiOutputClassifier(random_subspaces)

# Lista de modelos para comparar
models = {
    "Bagging_SVM": bagging_svm_multi,
    "Random_Subspaces": random_subspaces_multi
}

# Entrenamiento y evaluación
results = {}
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train_scaled, y_train)

    # Predicción
    y_pred = model.predict(X_test_scaled)

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Guardar resultados
    results[name] = {"accuracy": accuracy, "roc_auc":roc_auc , "report": report}
    print(f"=== {name} ===")
    print(f"Accuracy: {accuracy}")
    print(f"Roc-AUC: {roc_auc}")
    print("Classification Report:")
    print(report)

=== Bagging_SVM ===
Accuracy: 0.6721847930702599
Roc-AUC: 0.7943632909967411
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.77      0.64      1106
           1       0.77      0.78      0.78      2430

   micro avg       0.69      0.78      0.73      3536
   macro avg       0.66      0.78      0.71      3536
weighted avg       0.70      0.78      0.73      3536
 samples avg       0.36      0.38      0.36      3536



c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


=== Random_Subspaces ===
Accuracy: 0.6587102983638113
Roc-AUC: 0.7843039373471437
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.75      0.63      1106
           1       0.77      0.75      0.76      2430

   micro avg       0.68      0.75      0.71      3536
   macro avg       0.66      0.75      0.70      3536
weighted avg       0.70      0.75      0.72      3536
 samples avg       0.34      0.37      0.35      3536

